In [53]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [54]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\memba\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [55]:
#Scrape the [Mars News Site](https://redplanetscience.com/)
url = 'https://redplanetscience.com/'
browser.visit(url)
time.sleep(2)
html=browser.html

In [56]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [57]:
# Collect the latest News Title and Paragraph Text
latest_news_title=soup.find_all('div', class_='content_title')
latest_news_title = latest_news_title[0].text
paragraph_latest_news_title=soup.find_all('div', class_='article_teaser_body')
paragraph_latest_news_title = paragraph_latest_news_title[0].text
print(latest_news_title)
print('-----------------------------------------')
print(paragraph_latest_news_title)


NASA's Mars 2020 Rover Closer to Getting Its Name
-----------------------------------------
155 students from across the U.S. have been chosen as semifinalists in NASA's essay contest to name the Mars 2020 rover, and see it launch from Cape Canaveral this July.


In [58]:
# Visit the url for the Featured Space Image site (https://spaceimages-mars.com)
url_jpl = 'https://spaceimages-mars.com'
browser.visit(url_jpl)
time.sleep(2)
html_img=browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup_img = bs(html_img, 'html.parser')

In [59]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string
# to a variable called `featured_image_url`

images = soup_img.find_all('img', class_='headerimage fade-in')
for image in images:
    featured_image_url = (image['src'])

featured_image_url ='https://spaceimages-mars.com/' + featured_image_url
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [60]:
# Visit the Mars Facts webpage (https://galaxyfacts-mars.com) and use Pandas to scrape the table containing facts about
# the planet including Diameter, Mass, etc
url_facts = 'https://galaxyfacts-mars.com'
table = pd.read_html(url_facts)
table

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [61]:
df = table[0]
df.columns = ['Description', 'Mars','Earth']
df.set_index("Description", inplace=True)
df.head(10)

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [62]:
# Use Pandas to convert the data to a HTML table string.
html_table=df.to_html()
html_table = html_table.replace('\n','')
html_table=df.to_html('html_table.html')
html_table

In [63]:
# Visit the astrogeology site(https://marshemispheres.com/) to obtain high resolution images for each of Mar's hemispheres.
url_hemisp = 'https://marshemispheres.com/'
response = requests.get(url_hemisp)
soup_hemisp = bs(response.text, 'html.parser')

In [64]:
print(soup_hemisp)

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
<meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="css/main.css" media="screen" rel="stylesheet"/>
<link href="css/print.css" media="print" rel="stylesheet"/>
<link href="#" rel="icon" type="image/x-ico"/>
</head>
<body id="results">
<header>
<a href="#" style="float:right;margin-top:10px;" target="_blank">
<img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_2x.png"/>
</a>
<a href

In [65]:
links=[]
results = soup_hemisp.find_all('div', class_='description')

for result in results:
    to_visit = result.find('a','href', class_='itemLink product-item')['href']
    
    # visit hemisphere page
    browser.visit(f'{url_hemisp}{to_visit}')
    
    # create a soup object for the page being visited
    html_img = browser.html
    soup_ind = bs(html_img, 'html.parser')
    
    image_links = soup_ind.find('div', class_='wrapper')
    image_links = image_links.find('img',class_='wide-image')["src"]
    image_links = f'https://marshemispheres.com/{image_links}'
    
    title = soup_ind.find('h2',class_='title').text
    links.append({'title':title,'img_url':image_links})
    
    try:
        browser.links.find_by_text('back').click()
          
    except:
        pass

for i in range(len(links)-1):  
    print(f'{links[i]}, \n')
print(f'{links[len(links)-1]}')

{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, 

{'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, 

{'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, 

{'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}


In [66]:
browser.quit()